#### This is merely my second notebook and I have just started making them. Please do comment or give any suggestions that you can to help me learn and improve!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import the Libraries

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Let's specify the data paths!

In [ ]:
TRAIN_PATH = '../input/nlp-getting-started/train.csv'
TEST_PATH = '../input/nlp-getting-started/test.csv'
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

### Text Preprocessing

1. Tweets to lowercase

In [ ]:
for i in range(len(train)):
    train.text[i] = train.text[i].lower()

for i in range(len(test)):
    test.text[i] = test.text[i].lower()

2. Remove Stopwords

In [ ]:
tokenizer = nltk.tokenize.TreebankWordTokenizer()

In [ ]:
stop_words = stopwords.words('english')

for i in range(len(train)):
    tokens = tokenizer.tokenize(train.text[i])
    review = [i for i in tokens if not i in stop_words]
    train.text[i] = " ".join(review)
    
for i in range(len(test)):
    tokens = tokenizer.tokenize(test.text[i])
    review = [i for i in tokens if not i in stop_words]
    test.text[i] = " ".join(review)

3. I will be doing lemmatization for token normalization with TreeBank Tokenizer and WordNet Lemmatizer. I have tried various combinations of stemmer and lemmatizer but the best result that I have got is with only doing lemmatization

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
for i in range(len(train)):
    tokens = tokenizer.tokenize(train.text[i])
    train.text[i] = " ".join(lemmatizer.lemmatize(token) for token in tokens)

In [ ]:
for i in range(len(test)):
    tokens = tokenizer.tokenize(test.text[i])
    test.text[i] = " ".join(lemmatizer.lemmatize(token) for token in tokens)

In [ ]:
train.head()

We probably don't need the other 2 columns for this task, so I will drop the columns

In [ ]:
train = train.drop(labels=['keyword', 'location'], axis=1)
test = test.drop(labels=['keyword', 'location'], axis=1)

Now, we initialize the TFIDF vectorizer for creating the bag of n grams

In [ ]:
tfidf = TfidfVectorizer(min_df=5, max_df=0.5, ngram_range=(1, 2))

In [ ]:
fit_tfidf = tfidf.fit_transform(train.text).toarray()
train_data = pd.DataFrame(fit_tfidf, columns=tfidf.get_feature_names())

In [ ]:
test_data = pd.DataFrame(tfidf.transform(test.text).toarray(), 
                         columns=tfidf.get_feature_names())

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_data, 
                                  train.target, 
                                  test_size=0.2,
                                  random_state=47)

Create the Logistic Regression Object

In [ ]:
logreg = LogisticRegression()

Fit the object on the training data

In [ ]:
logreg.fit(x_train, y_train)

Make predictions on the validation and test data

In [ ]:
val_pred = logreg.predict(x_val)
y_pred = logreg.predict(test_data)

Let's check the score on validation data

In [ ]:
logreg.score(x_val, y_val)

We got an 80% probability score on the validation data, which is a decent score